<a href="https://colab.research.google.com/github/limkaram/Natural_language_processing_with_deep_learning/blob/main/Integer_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 정수 인코딩(Integer Encoding)
* 보통 단어 빈도수가 높은 순으로 내림차순 정렬한 후 정수 인코딩을 진행하는 경우가 많음

1. Dictionary 활용 정수 인코딩

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [ ]:
# 텍스트 정제 단계 : 불용어 제거, 길이 2 이하 단어 제거

sentences = sent_tokenize(text)
stop_words = set(stopwords.words('english'))
vocabulary = []
for sentence in sentences:
    words = word_tokenize(sentence)
    cleaned_words = []
    for word in words:
        word = word.lower()
        if word not in stop_words:
            if len(word) >= 2:
                cleaned_words.append(word)
    vocabulary.append(cleaned_words)
print(vocabulary)

In [ ]:
# dictionary '단어' : 빈도수 형태로 변환
vocap = {}

for vocaps in vocabulary:
    for v in vocaps:
        if v in vocap:
            vocap[v] += 1
        else:
            vocap[v] = 1

vocaps = sorted(vocap.items(), key=lambda x: x[1], reverse=True) # 내림차순 정렬
print(vocaps)

In [ ]:
# 정렬된 문자들에 인덱스 부여
word_to_index = {}
count = 0
for word, frequency in vocaps:
    if frequency > 1:  # 빈도수 2 이하 제거
        count += 1
        word_to_index[word] = count
word_to_index

In [ ]:
# 상위 빈도수 5개의 단어만 사용시
vocab_size = 5

words_frequency = [word for word, idx in word_to_index.items() if idx >= vocab_size + 1]
words_frequency

for word in words_frequency:
    del word_to_index[word]

# word_to_index를 상위 빈도수 5개의 단어로 축소하였으므로, 이에 포함되지 않는 단어는 'OOV(Out-Of_Vocabulary)'처리를 위해 OOV 추가
word_to_index['OOV'] = len(word_to_index) + 1
word_to_index

In [ ]:
# vocab_list : 문장 토큰화 후 각 문장들을 정제 후 단어 토큰화 시킨 결과 모음
encoded = []
for vocab in vocabulary:
    temp = []
    for word in vocab:
        if word not in word_to_index:
            temp.append(word_to_index['OOV'])
            continue
        temp.append(word_to_index[word])
    encoded.append(temp)
encoded

2. Counter 사용하기

In [ ]:
# from collections import Counter

words = sum(vocabulary, [])

counter = Counter(words)
vocab_size = 5
vocab = counter.most_common(vocab_size)

word_to_index = {}
i = 0
for word, frequency in vocab:
    i = i+1
    word_to_index[word] = i
print(word_to_index)

3. Keras 텍스트 전처리

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences=[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)  # 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스 부여
tokenizer.word_index  # 단어에 부여된 인덱스 확인
tokenizer.word_counts  # 단어별 빈도수 확인
tokenizer.texts_to_sequences(sentences)  # 입력으로 들어온 코퍼스에 대해 각 단어를 정해놓은 인덱스로 변환

In [ ]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1) # num_words 옵션 : 상위 5개 단어만 사용(Counter의 most_common과 동일)
tokenizer.fit_on_texts(sentences)

* num_words는 숫자를 0부터 카운트를 하므로 5개의 단어에 인덱스를 각각 부여했다면, 1~5까지의 인덱스 영역이 존재하고, 이를 반영해주기 위해선 num_words를 6으로해줘야 0~5까지 인덱스를 확인함
* 숫자 0에는 지정된 단어가 존재하지 않는데도 keras tokenizer 숫자까지 단어 집합의 크기로 산정하는 이유는 padding 때문

In [ ]:
tokenizer.word_index
tokenizer.word_counts
tokenizer.texts_to_sequences(sentences)

* tokenizer = Tokenizer(num_words = vocab_size + 1)를 적용해 주었음에도 tokenizer.word_index와 tokenizer.word_counts 출력은 인덱스가 1~5까지가 아닌 모든 단어를 출력함
* 신제 적용은 texts_to_sequences()에 적용이 되어 출력됨

In [ ]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tokenizer.fit_on_texts(sentences)

* 케라스 토크나이저는 기본적으로 단어 집합에 없는 단어인 OOV에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징을 가짐
* 단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용